<a href="https://colab.research.google.com/github/nguyenquangtrung8/quangtrung/blob/main/Playwright_batdongsan_com_vn_listing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Source: https://chatgpt.com/share/677c97a9-a51c-8006-8861-0ad4b46823ec

# Cài đặt

Cài playwright và driver để điều khiển trình duyệt bằng 2 câu lệnh dưới đây, lưu ý nếu trên máy có nhiều bản python thì cần chỉ rõ bản python muốn sử dụng, ví dụ

```
python3.10 -m pip install playwright
```

In [ ]:
!pip install playwright
!playwright install

# Chương trình chính

Chương trình dưới đây cần chạy trong chế độ giao diện đồ họa trên máy tính thay vì ẩn danh để có thể vượt được tường lửa của Cloudflare và lưu được trang html. Chương trình sẽ không thể chạy trong Google Colab.

1. Lưu đoạn mã dưới đây thành file python, ví dụ `test_bat_dong_san.py`.

2. Chạy đoạn mã

```
python test_bat_dong_san.py
```

In [ ]:
from playwright.sync_api import Playwright, sync_playwright
from pathlib import Path
from urllib.parse import urlparse
import time

# Function to visit pages and save their HTML content
def visit_pages(base_url: str, num_pages: int, timeout: int = 120000) -> None:
    with sync_playwright() as playwright:
        # Launch the browser in GUI mode
        browser = playwright.chromium.launch(headless=False)
        context = browser.new_context()
        page = context.new_page()

        # Loop through each page
        for i in range(1, num_pages + 1):
            url = base_url if i == 1 else f"{base_url}/p{i}"
            print(f"Visiting: {url}")

            # Retry mechanism for page load
            for attempt in range(3):
                try:
                    page.goto(url, wait_until="domcontentloaded", timeout=timeout)
                    break  # Break if successful
                except Exception as e:
                    print(f"Attempt {attempt + 1} failed: {e}")
                    if attempt == 2:
                        print(f"Skipping {url} after 3 failed attempts")
                        continue
                    time.sleep(5)  # Wait before retrying

            # Save HTML content
            html_content = page.content()
            save_html_to_file(url, html_content, i)

        # Close the browser
        context.close()
        browser.close()

# Helper function to save HTML content to a file
def save_html_to_file(url: str, content: str, page_number: int):
    # Parse the URL path
    path = urlparse(url).path.strip("/")
    # Replace slashes with hyphens to create a valid file name
    sanitized_path = path.replace("/", "-")
    # Create the file name
    filename = f"{sanitized_path}-p{page_number}.html"

    # Save the file in the html_pages directory
    output_dir = Path("html_pages")
    output_dir.mkdir(exist_ok=True)  # Create directory if it doesn't exist
    file_path = output_dir / filename
    with file_path.open("w", encoding="utf-8") as file:
        file.write(content)
    print(f"Saved: {file_path}")

# Main function
if __name__ == "__main__":
    # Input parameters
    base_url = "https://batdongsan.com.vn/ban-can-ho-chung-cu-haven-park-residences"
    num_pages = 5  # Number of pages to visit
    timeout = 120000  # Timeout in milliseconds (120 seconds)

    # Visit pages
    visit_pages(base_url, num_pages, timeout)


Bởi vì trang web batdongsan này có áp dụng cơ chế chặn truy cập bot nên cần tách công đoạn tải dữ liệu là file html và công đoạn đọc file html để tách thông tin rao bán riêng giúp hạn chế lỗi và xử lý từng phần. Sau khi chương trình hoàn thiện có thể gộp lại sau.

In [3]:
from bs4 import BeautifulSoup
import pandas as pd

# Path to the HTML file
html_file_path = "/content/ban-can-ho-chung-cu-haven-park-residences-p2-p2.html"

# Read the HTML file
with open(html_file_path, "r", encoding="utf-8") as file:
    html_content = file.read()

# Parse the HTML content
soup = BeautifulSoup(html_content, "html.parser")

# Extract property listings
property_listings = []

for listing in soup.find_all("a", class_="js__product-link-for-product-id"):
    # Extract data
    title = listing.get("title")
    price = listing.find("span", class_="re__card-config-price").get_text(strip=True) if listing.find("span", class_="re__card-config-price") else None
    area = listing.find("span", class_="re__card-config-area").get_text(strip=True) if listing.find("span", class_="re__card-config-area") else None
    price_per_m2 = listing.find("span", class_="re__card-config-price_per_m2").get_text(strip=True) if listing.find("span", class_="re__card-config-price_per_m2") else None
    bedrooms = listing.find("span", class_="re__card-config-bedroom").get_text(strip=True) if listing.find("span", class_="re__card-config-bedroom") else None
    toilets = listing.find("span", class_="re__card-config-toilet").get_text(strip=True) if listing.find("span", class_="re__card-config-toilet") else None
    location = listing.find("div", class_="re__card-location").get_text(strip=True) if listing.find("div", class_="re__card-location") else None
    description = listing.find("div", class_="re__card-description").get_text(strip=True) if listing.find("div", class_="re__card-description") else None
    agent_name = listing.find("div", class_="re__card-published-info-agent-profile-name").get_text(strip=True) if listing.find("div", class_="re__card-published-info-agent-profile-name") else None
    listing_date = listing.find("span", class_="re__card-published-info-published-at").get_text(strip=True) if listing.find("span", class_="re__card-published-info-published-at") else None

    # Append to list
    property_listings.append({
        "Title": title,
        "Price": price,
        "Area": area,
        "Price per m²": price_per_m2,
        "Bedrooms": bedrooms,
        "Toilets": toilets,
        "Location": location,
        "Description": description,
        "Agent Name": agent_name,
        "Listing Date": listing_date
    })

# Convert to DataFrame
df = pd.DataFrame(property_listings)

df

# # Save to CSV
# df.to_csv("property_listings.csv", index=False)


,Title,Price,Area,Price per m²,Bedrooms,Toilets,Location,Description,Agent Name,Listing Date
0,"Cần bán nhanh căn hộ 2PN, diện tích 68m2, đủ đ...","3,9 tỷ",68 m²,"57,35 tr/m²",2,2,"Xuân Quan, Văn Giang",None,‎Thanh Sang Chuyên Vinhomes,Đăng 6 ngày trước
1,Cần tiền bán gấp 3PN 94m2 Haven Park full đồ đ...,"4,9 tỷ",94 m²,"52,13 tr/m²",3,2,"Xuân Quan, Văn Giang",None,None,Đăng 6 ngày trước
2,Cần bán căn 2 ngủ Haven Park view BTĐ giá 3.15...,"3,15 tỷ",58 m²,"54,31 tr/m²",2,2,"Xuân Quan, Văn Giang",None,None,Đăng 6 ngày trước
3,Bán Penhouse Haven Park Ecopak view sông Hồng ...,24 tỷ,"265,4 m²","90,43 tr/m²",3,3,"Xuân Quan, Văn Giang",None,‎Nguyễn Thị Thảo,Đăng 1 tuần trước
4,Cần bán 2N 2WC Haven Park ban công Đông Nam. L...,"3,5 tỷ",68 m²,"51,47 tr/m²",2,2,"Xuân Quan, Văn Giang",None,None,Đăng 1 tuần trước
5,"Bán căn hộ H23x15 Haven Park, DT 102,5m2 căn g...","6,5 tỷ","102,5 m²","63,42 tr/m²",3,2,"Xuân Quan, Văn Giang",None,None,Đăng 1 tuần trước
6,Bán căn hộ mezza 58m2 tại Haven Park - Ecopark...,"4,7 tỷ",58 m²,"81,03 tr/m²",2,2,"Xuân Quan, Văn Giang",None,None,Đăng 1 tuần trước
7,Chính chủ bán căn hộ 63m2 - Haven Park 1 - 230...,"3,32 tỷ",63 m²,"52,7 tr/m²",2,2,"Xuân Quan, Văn Giang",None,None,Đăng 1 tuần trước
8,Bán căn hộ sân vườn HavenPark - Full nội thất ...,"2,45 tỷ",43 m²,"56,98 tr/m²",1,1,"Xuân Quan, Văn Giang",None,‎Chu Hoài Thương,Đăng 1 tuần trước
9,Căn hộ thông trần 3 phòng ngủ + 3 WC giá cực t...,"6,8 tỷ",100 m²,68 tr/m²,3,3,"Xuân Quan, Văn Giang",None,None,Đăng 1 tuần trước


In [6]:
from bs4 import BeautifulSoup
import pandas as pd
import io
from google.colab import files

def extract_property_data(html_content):
    """Extracts property data from HTML content."""
    soup = BeautifulSoup(html_content, "html.parser")
    property_listings = []

    for listing in soup.find_all("a", class_="js__product-link-for-product-id"):
        # Extract data
        title = listing.get("title")
        price = listing.find("span", class_="re__card-config-price").get_text(strip=True) if listing.find("span", class_="re__card-config-price") else None
        area = listing.find("span", class_="re__card-config-area").get_text(strip=True) if listing.find("span", class_="re__card-config-area") else None
        price_per_m2 = listing.find("span", class_="re__card-config-price_per_m2").get_text(strip=True) if listing.find("span", class_="re__card-config-price_per_m2") else None
        bedrooms = listing.find("span", class_="re__card-config-bedroom").get_text(strip=True) if listing.find("span", class_="re__card-config-bedroom") else None
        toilets = listing.find("span", class_="re__card-config-toilet").get_text(strip=True) if listing.find("span", class_="re__card-config-toilet") else None
        location = listing.find("div", class_="re__card-location").get_text(strip=True) if listing.find("div", class_="re__card-location") else None
        description = listing.find("div", class_="re__card-description").get_text(strip=True) if listing.find("div", class_="re__card-description") else None
        agent_name = listing.find("div", class_="re__card-published-info-agent-profile-name").get_text(strip=True) if listing.find("div", class_="re__card-published-info-agent-profile-name") else None
        listing_date = listing.find("span", class_="re__card-published-info-published-at").get_text(strip=True) if listing.find("span", class_="re__card-published-info-published-at") else None

        # Append to list
        property_listings.append({
            "Title": title,
            "Price": price,
            "Area": area,
            "Price per m²": price_per_m2,
            "Bedrooms": bedrooms,
            "Toilets": toilets,
            "Location": location,
            "Description": description,
            "Agent Name": agent_name,
            "Listing Date": listing_date
        })
    return property_listings

def main():
    uploaded = files.upload()

    all_listings = []
    for filename in uploaded.keys():
        try:
            html_content = uploaded[filename].decode('utf-8')
            listings = extract_property_data(html_content)
            all_listings.extend(listings)
        except Exception as e:
            print(f"Error processing file {filename}: {e}")

    if all_listings:
        df = pd.DataFrame(all_listings)
        display(df)

        # Option 1: Display DataFrame in Colab (already done with print(df))

        # Option 2: Download the CSV file
        if not df.empty:
            csv_filename = "combined_property_listings.csv"
            df.to_csv(csv_filename, index=False)
            files.download(csv_filename)
        else:
            print("No data to download.")

    else:
        print("No property listings found in the uploaded files.")

if __name__ == "__main__":
    main()

Saving ban-can-ho-chung-cu-haven-park-residences-p1.html to ban-can-ho-chung-cu-haven-park-residences-p1 (2).html
Saving ban-can-ho-chung-cu-haven-park-residences-p2-p2.html to ban-can-ho-chung-cu-haven-park-residences-p2-p2 (2).html
Saving ban-can-ho-chung-cu-haven-park-residences-p3-p3.html to ban-can-ho-chung-cu-haven-park-residences-p3-p3 (2).html
Saving ban-can-ho-chung-cu-haven-park-residences-p4-p4.html to ban-can-ho-chung-cu-haven-park-residences-p4-p4 (2).html
Saving ban-can-ho-chung-cu-haven-park-residences-p5-p5.html to ban-can-ho-chung-cu-haven-park-residences-p5-p5 (2).html


,Title,Price,Area,Price per m²,Bedrooms,Toilets,Location,Description,Agent Name,Listing Date
0,"Mình chủ nhà, Cần bán CHCC Cao Cấp( Ecopark- T...","8,5 tỷ","121,75 m²","69,82 tr/m²",None,None,"Xuân Quan, Văn Giang","Mình chủ nhà, Cần bán CHCC Cao Cấp( Ecopark- T...",‎Phạm Văn Hưng,Đăng 6 ngày trước
1,"Căn đẹp nhất tòa Haven Park, DT 30m2, giá chỉ ...","1,9 tỷ",30 m²,"63,3 tr/m²",None,None,"Xuân Quan, Văn Giang",Chính chủ cần bán căn Studio Haven Park đẹp nh...,‎Nguyễn Minh Đức,Đăng 1 tuần trước
2,Chính chủ cần bán gấp căn hộ 3 phòng ngủ góc -...,"4,9 tỷ","94,5 m²","51,85 tr/m²",3,2,"Xuân Quan, Văn Giang",Chính chủ gửi bán căn hộ 3 phòng ngủ căn góc c...,‎Tô Diệp,Đăng 1 tuần trước
3,Bán gấp studio Haven Park full đồ - tầng trung...,"1,8 tỷ",30 m²,60 tr/m²,1,1,"Xuân Quan, Văn Giang",Chính chủ gửi bán gấp căn studio Haven full nộ...,‎Chu Văn Nga,Đăng hôm nay
4,Bán căn hộ 101m2 view sân golf Aquabay - Thang...,Giá thỏa thuận,101 m²,None,3,2,"Xuân Quan, Văn Giang",None,‎Chu Hoài Thương,Đăng hôm nay
5,Chính chủ gửi bán siêu phẩm Mezza Havenpark tr...,"5,05 tỷ",116 m²,"43,53 tr/m²",None,None,"Xuân Quan, Văn Giang",None,None,Đăng hôm nay
6,"Bán 2PN 2WC Haven Park, giá bán 3,050 tỷ bao p...","3,05 tỷ",58 m²,"52,59 tr/m²",2,2,"Xuân Quan, Văn Giang",None,None,Đăng hôm nay
7,Bán căn hộ 68m2 thông thủy - view Đảo - 2 ngủ ...,"3,5 tỷ",68 m²,"51,47 tr/m²",2,2,"Xuân Quan, Văn Giang",None,None,Đăng hôm nay
8,"Bán căn Mezza 2 tầng, đã hoàn thiện toà Haven ...","7,1 tỷ",103 m²,"68,93 tr/m²",3,3,"Xuân Quan, Văn Giang",None,‎Phạm Tứ,Đăng hôm nay
9,Chính chủ cần bán căn 3PN Duplex Mezza thông t...,"6,8 tỷ",116 m²,"58,62 tr/m²",3,3,"Xuân Quan, Văn Giang",None,None,Đăng hôm qua


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>